<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big_Data_010_Trabalho_1_Auxilio_Emergencial_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-auxilio
*   Selecione a opção "Adicionar Atalho ao Drive"


In [1]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/auxilio/"

 auxilio_0.1pct.csv.gz
 auxilio_10pct.csv.gz
 auxilio_1pct.csv.gz
 auxilio.csv
 auxilio.csv.gz
'Dicionário de Dados - Auxílio Emergencial - Portal da transparência.pdf'
 estimativa_dou_2021.csv


In [8]:
!zcat "/content/drive/My Drive/auxilio/auxilio.csv.gz" | tail

"202011";"PR";"4126306";"SENGES";"12653005249";"***.340.289-**";"JOEL DE MELO";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"8ª";"Não há";"300,00"
"202011";"PR";"4126306";"SENGES";"00000000000";"***.094.989-**";"JOEL JUSTINO DOS SANTOS GABRIEL";"-2";"";"Não se aplica";"EXTRA CADUN";"8ª";"Não há";"300,00"
"202011";"PR";"4126306";"SENGES";"20081340049";"***.081.699-**";"JOEL PEREIRA ALVES";"20130961617";"***.688.569-**";"GISELE DE FATIMA DOS SANTOS";"BOLSA FAMILIA";"8ª";"";"194,00"
"202011";"PR";"4126306";"SENGES";"00000000000";"***.853.259-**";"JOEL RIBEIRO DE MORAES";"-2";"";"Não se aplica";"EXTRA CADUN";"8ª";"Não há";"300,00"
"202011";"PR";"4126306";"SENGES";"00000000000";"***.454.118-**";"JOELINE DE LIMA";"-2";"";"Não se aplica";"EXTRA CADUN";"7ª";"Não há";"600,00"
"202011";"PR";"4126306";"SENGES";"00000000000";"***.454.118-**";"JOELINE DE LIMA";"-2";"";"Não se aplica";"EXTRA CADUN";"6ª";"Não há";"600,00"
"202011";"PR";"4126306";"SENGES";"00000000000";"***.082.129-**";"JOELMA APARECIDA 

In [213]:
a = '202011;BA;;QUIXABEIRA;00000000000;***.560.975-**;JOSE NAILTON CUNHA DE OLIVEIRA;-2;;Não se aplica;EXTRA CADUN;7ª;Não há;300,00'

a = re.sub("\"", "", a)
first, second, third, *other, last = a.split(';')
print(last, second, third)


300,00 BA 


In [102]:
import re
print(last)
try:
  print(int(float(re.sub("[,]",".",re.sub("[.]", "",last)))))
except:
  print("falhou")

300,00
300


In [163]:
!cat "/content/drive/My Drive/auxilio/estimativa_dou_2021.csv" | grep "52,18391"

GO,52,18391,Professor Jamil,3211


In [167]:
popFile = open("/content/drive/My Drive/auxilio/estimativa_dou_2021.csv", "r")
popDict = {}
for line in popFile :
  try :
    uf,coduf,codmun,nome,pop = line.split(',')
    popDict[coduf+codmun] = [nome, int(pop)]
  except:
    pass      
popFile.close()

In [172]:
type(popDict["5218391"][1])

int

In [250]:
%%file auxilio.py
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol
import re
import logging

class Auxilio(MRJob):

  OUTPUT_PROTOCOL = TextProtocol

  def configure_args(self):
    super(Auxilio, self).configure_args()
    self.add_passthru_arg('--popFile', default="/content/drive/My Drive/auxilio/estimativa_dou_2021.csv")

  def mapper(self, _, line):
    try :
      # modificar para extrair os dados da linha e gerar saída conforme enunciado
      
      #---------------
      # Inicio Solucao
      #---------------
      #logging.warning("inicio: "+line)

      #Retira identificador de texto de aspas duplas
      line = re.sub("[\"]", "", line)
      #logging.warning("substitui aspas: \n"+line)

      #Retira tudo o que nao for letra, espaco e ponto e virgula
      #line = re.sub("[^a-z0-9\s;.]", "", line.lower())

      #Desempacota as variaveis
      pay_date, uf, cod_ibge, *other, pay_val = line.split(";")
      #logging.warning("codibge e valor: "+cod_ibge+"\t"+pay_val)

      #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
      # Tentativa de Try Except DanielSalvaNois
      if cod_ibge == '':
        raise ValueError("Codigo IBGE vazio")
      elif pay_val == '':
        raise ValueError("Valor pago vazio")
      #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

      #Transforma o valor pago em float
      pay_val = float(re.sub("[,]",".",re.sub("[.]", "", pay_val)))
      #logging.warning("codibge e valor transformado: "+cod_ibge+"\t"+pay_val)

      #Retorna a chave e o valor pago
      yield (cod_ibge, pay_val)

    except ValueError as error:
      logging.warning(error.args)
      logging.warning("linha descartada: "+line)
      pass

  def reducer_init(self):
    logging.warning("criando dicionário de municipios")
    popFile = open(self.options.popFile, "r")
    self.popDict = {}
    for line in popFile :
      try :
        uf,coduf,codmun,nome,pop = line.split(',')
        self.popDict[coduf+codmun] = [nome, int(pop)]
      except:
        pass      
    popFile.close()

  def reducer(self, key, values):
    try :
      # modificar para obter município e população a partir da chave, e 
      # calcular pagamentos per capita de acordo com o total de valores
      #---------------
      # Inicio Solucao
      #---------------
      total_val = sum(values)
      city, pop = self.popDict[key]
      yield key, str(total_val/pop)
    except :
      logging.warning("Deu ruim")
      pass

if __name__ == '__main__':
    logging.basicConfig(filename="log.txt")
    Auxilio.run()

Overwriting auxilio.py


In [254]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio_10pct.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211026.185520.774452
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211026.185520.774452/output
Streaming final output from /tmp/auxilio.root.20211026.185520.774452/output...
2304459	13.445278969957082
2304509	13.033321592109898
2304608	10.043646195442255
2304657	11.69032258064516
2304707	13.457930034439007
2304806	11.38064381270903
2304905	13.168553347000623
2304954	11.481741361098535
2305001	13.248185528212899
2305100	17.43899073526513
2305209	11.672065984298916
2305233	15.663040982665596
2305266	12.313111692192754
2305308	12.059288694615537
2305332	11.149332285938728
2305357	13.182083932022
2305407	11.40528234484576
2305506	12.904586376926268
2305605	10.548786074209803
2305654	10.921531430542382
2305704	9.633485248260975
2305803	13.343693650944292
2305902	13.10760823875578
2306009	12.472371263326597
2306108	13.110633946830266
2306207	11

In [ ]:
!cat log.txt

In [256]:
!rm log.txt

In [ ]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211026.185809.403984
Running step 1 of 1...


In [ ]:
drive.flush_and_unmount()